# Capacitated Facility Location Problem
Consider the Capacitated Facility Location Problem introduced in Chapter 7 of the Benders Decomposition lecture notes. An instance of the problem is provided below together with a class for the problem and a class for the full model.
- Q1 Solve the instance of the problem using Benders decomposition
- Q2 Compare its solution to that of the non-decomposed problem to ensure it is correct.

## Data

In [1]:
import random as r
n_locations = 15
n_customers = 30

r.seed(1)
# Random fixed costs between 100 and 300
fixed_costs = [(100 + (r.random() * 200)) for i in range(n_locations)]

# Random delivery costs between 10 and 40
delivery_costs = {(i,j):(10 + (r.random() * 30)) for j in range(n_customers) for i in range(n_locations)}

# Random demands between 50 and 100
demands = [(50 + (r.random() * 50)) for j in range(n_customers)]

# Random capacities between 100 and 140
capacities = [(150 + (r.random() * 50)) for i in range(n_locations)]


## Class for the Facility Location Problem.

In [2]:
class FacilityLocationProblem:

    def __init__(self,n_facilities,n_customers,fixed_costs,delivery_costs,demands,capacity):
        self.n_facilities = n_facilities
        self.n_customers = n_customers
        self.fixed_costs = fixed_costs
        self.delivery_costs = delivery_costs
        self.demands = demands
        self.capacity = capacity

We can now create an instance of the Facility Location Problem

In [3]:
flp = FacilityLocationProblem(n_locations, n_customers, fixed_costs, delivery_costs, demands, capacities)

## Class for the full model

In [4]:
class FullModel:

    def __init__(self, flp:FacilityLocationProblem):
        self.flp = flp
        self.m = Model()

        # Creates the variables
        self.y = self.m.addVars(flp.n_facilities, flp.n_customers, name="y")
        self.x = self.m.addVars(flp.n_facilities, vtype=GRB.BINARY, name="x")

        # Creates the objective
        self.m.setObjective(self.x.prod(flp.fixed_costs) + self.y.prod(flp.delivery_costs), GRB.MINIMIZE)

        # Constraints
        
        self.m.addConstrs(self.y.sum(i, '*') <= flp.capacity[i] * self.x[i] for i in range(flp.n_facilities))
        self.m.addConstrs(self.y.sum('*', j) >= flp.demands[j] for j in range(flp.n_customers))

    def solve(self):
        self.m.optimize()

    def print_solution(self):
        for i in range(self.flp.n_facilities):
            print('%s %g' % (self.x[i].varName, self.x[i].x))
        print('Obj: %g' % self.m.objVal)


# Solution

## Q1
We continue by implementing the Feasibility and Optimality subproblems

In [5]:
from gurobipy import Model, GRB

In [6]:
class FSP:

    def __init__(self, flp:FacilityLocationProblem, x:list):
        self.flp = flp
        self.m = Model()

        # Creates the variables
        y = self.m.addVars(flp.n_facilities, flp.n_customers, name="y")
        v1 = self.m.addVars(flp.n_facilities, name="v+")
        v2 = self.m.addVars(flp.n_customers, name="v-")

        # Creates the objective
        self.m.setObjective(v1.sum()+v2.sum(), GRB.MINIMIZE)

        # Constraints
        self.cc = self.m.addConstrs(y.sum(i, '*') - v1[i] <= flp.capacity[i] * x[i] for i in range(flp.n_facilities))
        self.dc = self.m.addConstrs(y.sum('*', j) + v2[j] >= flp.demands[j] for j in range(flp.n_customers))

    def solve(self):
        self.m.optimize()

    def get_results(self):

        dualsCC = self.m.getAttr(GRB.Attr.Pi, self.cc)
        dualsDC = self.m.getAttr(GRB.Attr.Pi, self.dc)

        return self.m.objVal, dualsCC, dualsDC



In [7]:
class OSP:

    def __init__(self, flp:FacilityLocationProblem, x:list):
        self.flp = flp
        self.m = Model()

        # Creates the variables
        y = self.m.addVars(flp.n_facilities, flp.n_customers, name="y")

        # Creates the objective
        self.m.setObjective(y.prod(flp.delivery_costs), GRB.MINIMIZE)

        # Constraints
        self.cc = self.m.addConstrs(y.sum(i, '*') <= flp.capacity[i] * x[i] for i in range(flp.n_facilities))
        self.dc = self.m.addConstrs(y.sum('*', j) >= flp.demands[j] for j in range(flp.n_customers))

    def solve(self):
        self.m.optimize()

    def get_results(self):

        dualsCC = self.m.getAttr(GRB.Attr.Pi, self.cc)
        dualsDC = self.m.getAttr(GRB.Attr.Pi, self.dc)

        return self.m.objVal, dualsCC, dualsDC

    def write(self):
        self.m.write("subproblem.lp")


We can now create the Master Problem

In [8]:
class Master:

    def __init__(self, flp:FacilityLocationProblem):
        self.flp = flp
        self.m = Model()

        # Creates the variables
        self.x = self.m.addVars(flp.n_facilities, vtype=GRB.BINARY, name="x")
        self.phi = self.m.addVar(name="phi")

        # Makes the variables visible in the callback
        self.m._x = self.x
        self.m._phi = self.phi

        # Creates the objective
        self.m.setObjective(self.phi + self.x.prod(flp.fixed_costs), GRB.MINIMIZE)

    def solve(self):

        def callback(model, where):
            # We want to run our callback ONLY
            # upon reaching a node in the Branch and Bound tree
            # with an integer solution. Gurobi calls this
            # condition GRB.Callback.MIPSOL.
            # Every time Gurobi calls our callback, it passes a value to the "where"
            # argument. Thus our callback code must run only when where indicates
            # that we are at an integer node (i.e., where is equal to GRB.Callback.MIPSOL).
            if where == GRB.Callback.MIPSOL:
                # Note that here we need to get the value
                # of the solution AT THE INTEGER NODE we have arrived at.
                # Observe how this value is retrieved, and that it is different
                # from what we learnt before (self.m.getAttr('x',x) or self.x.x).
                x_val = model.cbGetSolution(model._x)
                phi_val = model.cbGetSolution(model._phi)

                # Solves a FSP
                fsp = FSP(self.flp, x_val)
                fsp.solve()
                obj, dualsCC, dualsDC = fsp.get_results()               
                # Feasibility test
                if obj > 0:
                    # Can you write the cut in a better way?
                    lhs = 0
                    for i in range(self.flp.n_facilities):
                        lhs = lhs + dualsCC[i] * self.flp.capacity[i] * model._x[i]
                    for j in range(self.flp.n_customers):
                        lhs = lhs + dualsDC[j] * self.flp.demands[j]
                    model.cbLazy(lhs <= 0)
                    print("Added an FC")
                else:
                    # Solves an OSP
                    osp = OSP(self.flp, x_val)
                    osp.solve()

                    obj, dualsCC, dualsDC = osp.get_results()

                    # Optimality test
                    if phi_val >= obj:
                        print("Optimal solution found")
                    else:
                        lhs = model._phi
                        rhs = 0
                        for i in range(self.flp.n_facilities):
                            rhs = rhs + dualsCC[i] * self.flp.capacity[i] * model._x[i]
                        for j in range(self.flp.n_customers):
                            rhs = rhs + dualsDC[j] * self.flp.demands[j]
                        model.cbLazy(lhs >= rhs)
                        print("Added an OC")

        # We inform Gurobi that we are going to pass
        # Lazy Constraints.
        self.m.setParam(GRB.Param.LazyConstraints, 1)
        # We pass the callback to the optimize method.
        self.m.optimize(callback)

    def print_solution(self):
        for i in range(self.flp.n_facilities):
            print('%s %g' % (self.x[i].varName, self.x[i].x))
        print('Obj: %g' % self.m.objVal)

We solve the problem using BD

In [9]:
mp = Master(flp)
mp.solve()

Restricted license - for non-production use only - expires 2023-10-25
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 0 rows, 16 columns and 0 nonzeros
Model fingerprint: 0xda78efce
Variable types: 1 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 45 rows, 495 columns and 945 nonzeros
Model fingerprint: 0xe849b585
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+02]
Presolve removed 45 rows and 495 columns
Presolve time: 0.02s
Pr

## Q2

Finally, we compare the solution to that of the non-decomposed model

In [10]:
m = FullModel(flp)
m.solve()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 45 rows, 465 columns and 915 nonzeros
Model fingerprint: 0xcfdf1a6a
Variable types: 450 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
Found heuristic solution: objective 57844.422792
Presolve time: 0.00s
Presolved: 45 rows, 465 columns, 915 nonzeros
Variable types: 450 continuous, 15 integer (15 binary)

Root relaxation: objective 2.960215e+04, 56 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 29602.1475    0    3 57844.4228 29602.1475  48.8%     -    0s
H    0     0                    29806.453962 29602.1475  0.69%     -   

In [11]:
mp.print_solution()
m.print_solution()

x[0] 1
x[1] 1
x[2] 1
x[3] 1
x[4] 1
x[5] 1
x[6] 1
x[7] 1
x[8] 1
x[9] -0
x[10] 1
x[11] 1
x[12] 1
x[13] 1
x[14] 1
Obj: 29700.8
x[0] 1
x[1] 1
x[2] 1
x[3] 1
x[4] 1
x[5] 1
x[6] 1
x[7] 1
x[8] 1
x[9] 0
x[10] 1
x[11] 1
x[12] 1
x[13] 1
x[14] 1
Obj: 29700.8
